In [3]:
import json
from twython import Twython
import pandas as pd
import numpy as np


# Enter your keys/secrets as strings in the following fields
#credentials = {}
#credentials['CONSUMER_KEY'] = ''
#credentials['CONSUMER_SECRET'] = ''
#credentials['ACCESS_TOKEN'] = '-'
#credentials['ACCESS_SECRET'] = ''

# Save the credentials object to file
#with open("twitter_credentials.json", "w") as file:
#    json.dump(credentials, file)
    
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file) 
    
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])    

## look for key word

In [ ]:
query = {'q': 'energy',
        'result_type': 'recent',  # other options 'mixed','popular','recent'
        'count': 10,   # max 100
         # 'until':"2019-02-01",
         'geocode': '51.51753,-0.11214,1000mi'
        }

dict_ = {'user': [],
         'date': [],
         'text': [],
         'favorite_count': [],
         'retweet_count': [],
         'user_loc': [],
         'followers_count': [],
         'lang': [],
         'user_desc': [],
         'hashtags': [],
         'user_verified': [],
         'geo': [],
         'coordinates':[],
         'place':[],
         #'user_loc_country':[]
         
        }


for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['user_loc'].append( status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['lang'].append(status['lang'])
    dict_['user_desc'].append(status['user']['description'])
    dict_['user_verified'].append(status['user']['verified'])
    dict_['hashtags'].append(status['entities']['hashtags'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['geo'].append(status['geo'])
    dict_['coordinates'].append(status['coordinates'])
    dict_['place'].append(status['place'])
    #dict_['user_loc_country'].append(status['user']['derived']['locations']['country'])
    
df = pd.DataFrame(dict_)
#dict_['text'][0]
df

In [ ]:
df.shape

In [ ]:
df

In [ ]:
#cities = pd.read_csv('list_cities.csv',sep=';')

## Translate queries and Tweets

In [ ]:
from os import environ

from google.cloud import translate

def gtrans(text,dest='en'):
    
    '''this function represents the google translate API. use wisely!
    its expensive (20$/1mio characters, makes only 5000 tweets)'''
    
    
    project_id = 'lewagon-bootcamp-timwolfram' #environ.get("PROJECT_ID", "")

    parent = f"projects/{project_id}"
    client = translate.TranslationServiceClient()


    sample_text = text
    target_language_code = dest

    response = client.translate_text(
        contents=[sample_text],
        target_language_code=target_language_code,
        parent=parent,
    )

    for translation in response.translations:
        output_trans = translation.translated_text
        
    return output_trans

In [ ]:
gtrans('Hallo Welt')

In [ ]:
dict_ = {'user': [],
             'date': [],
             'text': [],
             'text_en': [],
             'favorite_count': [],
             'retweet_count': [],
             'user_loc': [],
             'followers_count': [],
             'lang': [],
             'user_desc': [],
             'user_desc_en': [],
             'hashtags': [],
             'user_verified': [],

             #'geo': [],
             #'coordinates':[],
             #'place':[]

             #'user_loc_country':[]

            }



query_word = 'energy'

lang_list = ['en','de','fr','el','it','es',
                'pl', 'ro', 'nl', 'hu','pt',
                'sv',  'cs', 'bg', 'sk', 'da',
                'fi', 'hr', 'lt'
            ]

count_tweets = 1
result_type = 'mixed'

for lang in lang_list:
    
    query_word = gtrans(query_word, dest=lang)
    if lang == 'en':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': '51.51753,-0.11214,1000mi'
            }
        
    elif lang == 'fr':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': '47.22283,2.07099,1000mi'
            }
    elif lang == 'es':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': ' 40.42955,-3.67930,1000mi'
            }
        
    elif lang == 'pt':
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
            'geocode': '39.82059,-7.49342,1000mi'
            }
        
    else:
        query = {'q': query_word,
            'result_type': result_type,  # other options 'mixed','popular','recent'
            'count': count_tweets,   # max 100
             # 'until':"2019-02-01",
             #'geocode': '50.0598058,14.3255426,1000km'
            }

   
    for status in python_tweets.search(**query)['statuses']:
        
        dict_['user'].append(status['user']['screen_name'])
        dict_['date'].append(status['created_at'])
        dict_['text'].append(status['text'])
        dict_['text_en'].append(gtrans(str(status['text']), dest='en'))#src=str(status['lang'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['user_loc'].append( status['user']['location'])
        dict_['followers_count'].append(status['user']['followers_count'])
        dict_['lang'].append(status['lang'])
        dict_['user_desc'].append(status['user']['description'])
        try:
            dict_['user_desc_en'].append(gtrans(str(status['user']['description']), dest='en'))
        except:
            dict_['user_desc_en'].append('none')
        dict_['user_verified'].append(status['user']['verified'])
        dict_['hashtags'].append(status['entities']['hashtags'])
        dict_['retweet_count'].append(status['retweet_count'])


        #dict_['geo'].append(status['geo'])
        #dict_['coordinates'].append(status['coordinates'])
        #dict_['place'].append(status['place'])
        #dict_['user_loc_country'].append(status['user']['derived']['locations']['country'])


df = pd.DataFrame(dict_)
#dict_['text']
df


In [ ]:
#cities = pd.read_csv('list_cities2.csv', delimiter=';')

In [ ]:
#cities

In [ ]:
### filter for european countries

## sentiment analysis

In [ ]:
from textblob import TextBlob
import nltk
import string
#nltk.download('brown')
import string

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [ ]:
text = df['text_en'][1]
text

In [ ]:
#text = text.lower()

##remove punctiation

for punctuation in string.punctuation:
    text = text.replace(punctuation, '') 
    
##remove stopwords
stop_words = set(stopwords.words('english')) 

word_tokens = word_tokenize(text) 
  
text = [w for w in word_tokens if not w in stop_words] 

#text = ' '.join(text)
  

In [ ]:
text = ' '.join(text)
text

In [ ]:
wiki = TextBlob(text)
#wiki.tags

In [ ]:
print(wiki.noun_phrases)

In [ ]:
testimonial = TextBlob(text)
print(testimonial.sentiment)
print(testimonial.sentiment.polarity)

## all functions for .py file

In [ ]:
from os import environ
import json
import pandas as pd
from google.cloud import translate
from twython import Twython


# Enter your keys/secrets as strings in the following fields
#credentials = {}
#credentials['CONSUMER_KEY'] = ''
#credentials['CONSUMER_SECRET'] = ''
#credentials['ACCESS_TOKEN'] = '-'
#credentials['ACCESS_SECRET'] = ''

#Save the credentials object to file
#with open("twitter_credentials.json", "w") as file:
#    json.dump(credentials, file)
    
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file) 
    
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])    


def get_places():
    
    '''creates a list of places and countries in europe, for locality-filtering the output tweets'''
    cities = pd.read_csv('list_cities3.csv',delimiter=';')
    
    list_cities = list(cities['city'])
    list_cities = [element.lower() for element in list_cities]
    list_countries = list(set(cities['county']))
    list_countries = [element.lower() for element in list_countries]
    list_eur = list_cities + list_countries
    return list_eur


def gtrans(text,dest='en'):
    
    '''this function represents the google translate API. use wisely!
    its expensive (20$/1mio characters, makes only 5000 tweets)'''
    
    
    project_id = 'lewagon-bootcamp-timwolfram' #environ.get("PROJECT_ID", "")

    parent = f"projects/{project_id}"
    client = translate.TranslationServiceClient()


    sample_text = text
    target_language_code = dest

    response = client.translate_text(
        contents=[sample_text],
        target_language_code=target_language_code,
        parent=parent,
    )

    for translation in response.translations:
        output_trans = translation.translated_text
        
    return output_trans


def remove_duplicates(tweets):
    li = []
    for tweet in tweets:
        if not tweet in li:
            li.append(tweet)

    return li


def get_tweets(query,count,result_type='mixed'):   #count: max 10 because its looking for all 19 languages at once

    '''this function uses the twitter search endpoint with max 190 requests/15 min
    the output is a list of dictionaries with one dict per tweet.
    keys are the features of the tweets'''
    
    ''' it uses the google-translate API wich kosts 20$ per ~5000 tweets '''

    query_word = query
    count_tweets = count
    
    lang_list = ['en','de','fr','el','it','es',
                    'pl', 'ro', 'nl', 'hu','pt',
                    'sv',  'cs', 'bg', 'sk', 'da',
                    'fi', 'hr', 'lt'
                ]

    #dict_ = {}
    
    result_type = result_type # other options 'mixed','popular','recent'

    list_tweets = []

    for lang in lang_list:

        query_word = gtrans(query_word, dest=lang)
        if lang == 'en':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '51.51753,-0.11214,1000mi',
                'tweet_mode': 'extended'
                }

        elif lang == 'fr':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '47.22283,2.07099,1000mi',
                'tweet_mode': 'extended'
                }
            
        elif lang == 'es':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '40.42955,-3.67930,1000mi',
                'tweet_mode': 'extended'
                }

        elif lang == 'pt':
            query = {'q': query_word,
                'result_type': result_type,  
                'count': count_tweets,   # max 100
                'geocode': '39.82059,-7.49342,1000mi',
                'tweet_mode': 'extended'
                }

        else:
            query = {'q': query_word,
                'result_type': result_type,  # 
                'count': count_tweets,   # max 100
                #'geocode': '50.0598058,14.3255426,1000km'
                'tweet_mode': 'extended'
                }


        for status in python_tweets.search(**query)['statuses']:
            dict_ = {}
            dict_['id'] = status['id']
            dict_['user'] = status['user']['screen_name']
            dict_['date'] = status['created_at']
            dict_['text'] = status['full_text']
            dict_['text_en'] = gtrans(str(status['full_text']), dest='en')
            dict_['favorite_count'] = status['favorite_count']
            dict_['user_loc'] =  status['user']['location']
            dict_['followers_count'] = status['user']['followers_count']
            dict_['lang'] = status['lang']
            dict_['user_desc'] = status['user']['description']
            try:
                dict_['user_desc_en'] = gtrans(str(status['user']['description']), dest='en')
            except:
                dict_['user_desc_en'] = 'none'
            dict_['user_verified'] = status['user']['verified']
            dict_['hashtags'] = status['entities']['hashtags']
            dict_['retweet_count'] = status['retweet_count']

            list_tweets.append(dict_) 
    
    return list_tweets




In [55]:
from geopy import geocoders
from geopy.geocoders import Nominatim
import math


def get_country(city):
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    country = df_europe.loc[df_europe['city'] == city, 'country'].iloc[0]
    return country


def region_lat(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return latitude


def region_lon(region):
    geolocator = Nominatim(user_agent='myapplication')
    try:
        location = geolocator.geocode(region)
        latitude = location[1][0]
        longitude = location[1][1]
    except:
        latitude = 'NaN'
        longitude = 'NaN'
        
    return longitude


def count_cities(tweets):
    
    #in the user locations sometimes its Paris, France, sometimes France
    # sometimes Paris, here we try to filter this for cities
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    
    city_counts = {
    
    'city': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[0] in list_cities:
                city_counts['city'].append(list_loc[0])
                city_counts['likes'].append(tweet['favorite_count'])
                city_counts['tweets'].append(1)
                city_counts['retweets'].append(tweet['retweet_count'])
                city_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_cities:
            city_counts['city'].append(tweet['user_loc'])
            city_counts['likes'].append(tweet['favorite_count'])
            city_counts['tweets'].append(1)
            city_counts['retweets'].append(tweet['retweet_count'])
            city_counts['sentiment'].append(tweet['compound_score'])
            
            
            
    return city_counts


    
def count_countries(tweets):
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    country_counts = {
    
    'country': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[1] in list_countries:
                country_counts['country'].append(list_loc[1])
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])
                
            if list_loc[0] in list_cities:
                country_counts['country'].append(get_country(list_loc[0])) #translates city to country
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])

            
            
        elif tweet['user_loc'] in list_cities:
            country_counts['country'].append(get_country(tweet['user_loc']))
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_countries:
            country_counts['country'].append(tweet['user_loc'])
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
    return country_counts



def clean_and_sum(df, region):
    df = df.dropna()
    df['sentiment'] = df['sentiment'] * (df['retweets']+1) #scores sentiment of a tweet by retweet_count
    df = df.groupby(by=region, as_index=False).sum()
    df['sentiment'] = df['sentiment'] / df['tweets']
    return df


def add_lat_lon(df):
    df['lat'] = df['country'].apply(region_lat)
    df['lon'] = df['country'].apply(region_lon)
    return df


def add_radius(df):
    df["radius"] = df["retweets"].apply(lambda likes: math.sqrt(likes)*1000 + 10000)
    return df


##### define colors from sentiment ####

def sent_ref(df):
    df['sent_ref'] = df['sentiment'] / df['tweets']
    return df

def sent_color_green(sent):
    
    if sent > 0:
        green = 100
    else:
        green = 0
        
    return green


def sent_color_red(sent):
    
    if sent < 0:
        red = 100
    else:
        red = 0
    
    return red
    
    
def sent_color_blue(sent):
    blue = sent * 0
    return blue


def sent_shade(sent):
    if sent < 0:
        shade = -300 * sent + 50
    else:
        shade = 300 * sent + 50
    return shade
    
def sent_colors(df):
    df['r'] = df['sent_ref'].apply(sent_color_red)
    df['g'] = df['sent_ref'].apply(sent_color_green)
    df['b'] = df['sent_ref'].apply(sent_color_blue)
    df['s'] = df['sent_ref'].apply(sent_shade)
    df['sent_ref'] = df['sent_ref'].round(decimals=4)
    return df


## geographical mapping of tweets

In [47]:
#this cell creates a dataframe with european countries with lat and lon

#df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
#list_countries = list(df_europe['countrie'])
#countries_df = pd.DataFrame(list_countries)
#countries_df = countries_df.drop_duplicates()
#countries_df = pd.DataFrame(countries_df)
#countries_df = countries_df.rename(columns={0: 'country'})
#countries_df['lat'] = countries_df['country'].apply(region_lat)
#countries_df['lon'] = countries_df['country'].apply(region_lon)
#countries_df.to_csv('country_loc.csv')
#countries_df

In [46]:
#this cell creates a csv-file with europan cities with lat and lon

#df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
#list_cities = list(df_europe['city'])
#cities_df = pd.DataFrame(list_cities)
#cities_df = cities_df.drop_duplicates()
#cities_df = pd.DataFrame(cities_df)
#cities_df = cities_df.rename(columns={0: 'city'})
#cities_df['lat'] = cities_df['city'].apply(region_lat)
#cities_df['lon'] = cities_df['city'].apply(region_lon)
#cities_df.to_csv('city_loc.csv')
#cities_df

In [56]:
city_counts = count_cities(data_dict)
df_city_counts = pd.DataFrame(city_counts)
#df_city_counts.head()
df_city_counts.shape
df_city_counts = clean_and_sum(df_city_counts)
df_city_counts.head()

,city,tweets,likes,retweets,sentiment
0,Alicante,1,0,3535,0.000000
1,Athens,1,0,0,0.000000
2,Banja Luka,1,0,0,0.636900
3,Barcelona,3,0,0,0.300467
4,Belgrade,14,36,712,12.316807


In [45]:
city_counts_merged = df_city_counts.merge(cities_df, how='left', on='city')
city_counts_merged

,city,tweets,likes,retweets,sentiment,lat,lon
0,Alicante,1,0,3535,0.0000,38.353738,-0.490185
1,Athens,1,0,0,0.0000,37.983941,23.728305
2,Banja Luka,1,0,0,0.6369,44.772084,17.191765
3,Barcelona,3,0,0,0.9014,41.382894,2.177432
4,Belgrade,14,36,712,5.4424,44.817813,20.456897
5,Bochum,1,0,0,0.5574,51.481811,7.219664
6,Bonn,1,0,0,0.6115,50.735851,7.10066
7,Bordeaux,2,0,18,0.7163,44.841225,-0.580036
8,Brno,1,0,0,0.5661,49.192244,16.611338
9,Bucharest,1,0,0,0.6369,44.436141,26.10272


In [48]:
df = city_counts_merged

In [ ]:
def score_sent(df):
    df['score_sent'] = df['sentiment'] * df['retweets']
    return df
    

In [24]:
country_counts = count_countries(data_dict)
df_country_counts = pd.DataFrame(country_counts)
#df_country_counts.head()
#df_country_counts.shape


(123, 5)

In [41]:
import math

def clean_and_sum(df):
    
    df = df.dropna()
    df = df.groupby(by="city", as_index=False).sum()
    return df

def add_lat_lon(df):
    df['lat'] = df['country'].apply(region_lat)
    df['lon'] = df['country'].apply(region_lon)
    return df

def add_radius(df):

    df["radius"] = df["retweets"].apply(lambda likes: math.sqrt(likes)*1000 + 10000)
    return df



In [28]:
df_country_counts = clean_and_sum(df_country_counts)
df_country_counts

,country,tweets,likes,retweets,sentiment
0,Belgium,3,0,1,0.6313
1,Bosnia,1,0,0,0.6369
2,Bulgaria,13,193,7,5.3410
3,Czech Republic,4,0,1,2.1757
4,Denmark,7,20,13,1.0772
5,Finland,20,19,87,5.3187
6,France,15,2759,1031,2.2519
7,Germany,6,4,2,0.9962
8,Greece,6,1255,224,2.1663
9,Hungary,1,1,0,-0.8573


In [31]:
counts_merged = df_country_counts.merge(countries_df, how='left', on='country')

In [32]:
counts_merged

,country,tweets,likes,retweets,sentiment,lat,lon
0,Belgium,3,0,1,0.6313,50.640281,4.666715
1,Bosnia,1,0,0,0.6369,NaN,NaN
2,Bulgaria,13,193,7,5.3410,42.607397,25.485662
3,Czech Republic,4,0,1,2.1757,49.816700,15.474954
4,Denmark,7,20,13,1.0772,55.670249,10.333328
5,Finland,20,19,87,5.3187,63.246778,25.920916
6,France,15,2759,1031,2.2519,46.603354,1.888334
7,Germany,6,4,2,0.9962,51.083420,10.423447
8,Greece,6,1255,224,2.1663,38.995368,21.987713
9,Hungary,1,1,0,-0.8573,47.181759,19.506094


In [33]:
print('Serbia' in list_countries)

False


In [154]:
df_country_counts.to_csv('country_counts.csv')

In [110]:
city_counts = count_cities(data_dict)
df_city_counts = pd.DataFrame(city_counts)
df_city_counts.head()
#df_city_counts.shape

,city,tweets,likes,retweets
0,Bucharest,1,0,0
1,Bratislava,1,0,0
2,Thessaloniki,1,0,3
3,Bratislava,1,17,4
4,Zagreb,1,18,0


In [106]:
# drop rows with user_loc as city-name

#df_city_counts = df_city_counts[(df_city_counts != 'user_loc').all(axis=1)]
#df_city_counts = df_city_counts[(df_city_counts != 'NaN').all(axis=1)]
df_city_counts.head()

,city,tweets,likes,retweets


In [28]:
df_city_counts = df_city_counts.dropna()
df_city_counts = df_city_counts.groupby(by="city", as_index=False).sum()

df_city_counts['lat'] = df_city_counts['city'].apply(region_lat)
df_city_counts['lon'] = df_city_counts['city'].apply(region_lon)
#df_city_counts["radius"] = df_city_counts["tweets"].apply(lambda likes: math.sqrt(likes)*1000 + 10000)

df_city_counts

,city,tweets,likes,retweets,lat,lon
0,3 Rawlinson St,1,0,0,-27.469474,153.097644
1,Brussels,1,0,1,50.846557,4.351697
2,Earth,1,0,1,34.233137,-102.410749
3,Gironde,1,0,3,44.883746,-0.605126
4,Kampala,1,0,1,0.317714,32.581354
5,Kolkata,1,0,6,22.541418,88.357691
6,Limbo,1,625,221,5.224918,9.773734
7,Los Angeles,1,3432,871,34.053691,-118.242766
8,Nairobi,3,0,5,-1.303169,36.826061
9,Nanyuki,1,0,12,0.014030,37.074632


In [ ]:
df_city_counts.to_csv('city_counts.csv')


In [193]:
def sent_ref(df):
    df['sent_ref'] = df['sentiment'] / df['tweets']
    return df

def sent_color_green(sent):
    
    if sent > 0:
        green = 100
    else:
        green = 0
        
    return green


def sent_color_red(sent):
    
    if sent < 0:
        red = 100
    else:
        red = 0
    
    return red
    
    
def sent_color_blue(sent):
    blue = sent * 0
    return blue


def sent_shade(sent):
    if sent < 0:
        shade = -300 * sent + 50
    else:
        shade = 300 * sent + 50
    return shade



In [194]:
def sent_colors(df):
    df['r'] = df['sent_ref'].apply(sent_color_red)
    df['g'] = df['sent_ref'].apply(sent_color_green)
    df['b'] = df['sent_ref'].apply(sent_color_blue)
    df['s'] = df['sent_ref'].apply(sent_shade)
    df['sent_ref'] = df['sent_ref'].round(decimals=4)
    return df


## Database retrieve

In [17]:
import pandas as pd
import math
import requests


def get_tweets(query,source):

  '''sources: twitter_query, twitter_politicians, twitter_press'''

  params=dict(q=query,limit=100000)
  headers={'X-Meili-API-Key':'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}
  lexai_url = f"http://35.223.18.2/indexes/{source}/search/"
  data = requests.get(lexai_url,params=params,headers=headers).json()
  data_df=pd.DataFrame(data['hits'])
  data_df = data_df[data_df["timestamp"] >= 1.622115e+09]   ###filters tweets which are newer than certain timepoint ~27th may
  data_df = data_df.sort_values(by=['timestamp'])
  data_dict = data_df.to_dict('records')   #creates dictionary for further use
  return data_dict


def clean_and_sum(df, region):
    df = df.dropna()
    df['sentiment'] = df['sentiment'] * (df['retweets']+1) #scores sentiment of a tweet by retweet_count
    df = df.groupby(by=region, as_index=False).sum()
    df['sentiment'] = df['sentiment'] / df['tweets']
    df['sentiment'] = df['sentiment'].round(decimals=1)

    return df


def get_country(city):

    #takes a city as input and give the assigned country as output

    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    country = df_europe.loc[df_europe['city'] == city, 'country'].iloc[0]
    return country


def count_cities(tweets):
    
    #in the user_location-column, sometimes its Paris, France, sometimes France
    # sometimes Paris, here we try to filter this for cities
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    
    city_counts = {
    'city': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[0] in list_cities:
                city_counts['city'].append(list_loc[0])
                city_counts['likes'].append(tweet['favorite_count'])
                city_counts['tweets'].append(1)
                city_counts['retweets'].append(tweet['retweet_count'])
                city_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_cities:
            city_counts['city'].append(tweet['user_loc'])
            city_counts['likes'].append(tweet['favorite_count'])
            city_counts['tweets'].append(1)
            city_counts['retweets'].append(tweet['retweet_count'])
            city_counts['sentiment'].append(tweet['compound_score'])
            
    df_city_counts = pd.DataFrame(city_counts)
    df_city_counts = clean_and_sum(df_city_counts, 'city')

    return df_city_counts


    
def count_countries(tweets):
    
    df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
    list_cities = list(df_europe['city'])
    list_countries = list(df_europe['country'])
    
    country_counts = {
    
    'country': [],
    'tweets':[],
    'likes': [],
    'retweets': [],
    'sentiment': []
    }
    
    
    for tweet in tweets:
        if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[1] in list_countries:
                country_counts['country'].append(list_loc[1])
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])
                
            if list_loc[0] in list_cities:
                country_counts['country'].append(get_country(list_loc[0])) #translates city to country
                country_counts['likes'].append(tweet['favorite_count'])
                country_counts['tweets'].append(1)
                country_counts['retweets'].append(tweet['retweet_count'])
                country_counts['sentiment'].append(tweet['compound_score'])

            
            
        elif tweet['user_loc'] in list_cities:
            country_counts['country'].append(get_country(tweet['user_loc']))
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
        elif tweet['user_loc'] in list_countries:
            country_counts['country'].append(tweet['user_loc'])
            country_counts['likes'].append(tweet['favorite_count'])
            country_counts['tweets'].append(1)
            country_counts['retweets'].append(tweet['retweet_count'])
            country_counts['sentiment'].append(tweet['compound_score'])
            
    df_country_counts = pd.DataFrame(country_counts)
    df_country_counts = clean_and_sum(df_country_counts, 'country')
    
    return df_country_counts
 
   
def add_radius(df):
    df["radius"] = df["retweets"].apply(lambda likes: math.sqrt(likes)*1000 + 20000)
    return df


##### define colors from sentiment ####


def sent_color_red(sent):
    
    if sent > 0:
        red = 96
    else:
        red = 115
    
    return red
    

def sent_color_green(sent):
    
    if sent > 0:
        green = 130
    else:
        green = 31
        
    return green

    
def sent_color_blue(sent):
    if sent > 0:
        blue = 253
    else:
        blue = 125
    
    return blue


def sent_shade(sent):
    if sent < 0:
        shade = 50 * math.sqrt(-sent) + 50
    else:
        shade = 50 * math.sqrt(sent) + 50
    return shade
    
    
def sent_colors(df):
    df['r'] = df['sentiment'].apply(sent_color_red)
    df['g'] = df['sentiment'].apply(sent_color_green)
    df['b'] = df['sentiment'].apply(sent_color_blue)
    df['s'] = df['sentiment'].apply(sent_shade)
    df['sentiment'] = df['sentiment'].round(decimals=4)
    return df


def refine_cities(data_dict):

    #this function concludes the steps of summing the tweet counts, likes etc. 
    #by city in a dataframe and assign them to a latitude and longitude
    #to display the data on a geographical map
    #the data_dict argument must be a list of dictionaries, which is the output of the
    #get_tweets function

    df_cities_loc = pd.read_csv('city_loc.csv')
    

    df_cities = count_cities(data_dict)
    df_cities = df_cities.merge(df_cities_loc, how='left', on='city')
    df_cities = df_cities.dropna()
    df_cities = add_radius(df_cities)
    df_cities = sent_colors(df_cities)
    return df_cities


def refine_countries(data_dict):

    #does the same as refine_cities just for countries

    df_countries_loc = pd.read_csv('country_loc.csv')

    df_countries = count_countries(data_dict)
    df_countries = df_countries.merge(df_countries_loc, how='left', on='country')
    df_countries = df_countries.dropna()
    df_countries = add_radius(df_countries)
    df_countries = sent_colors(df_countries)
    return df_countries

def refine_pol_press(data_dict):

    #this function concludes the steps of summing the tweet counts, likes etc. 
    #by city in a dataframe and assign them to a latitude and longitude
    #to display the data on a geographical map
    
    df = pd.DataFrame(data_dict)

    df = df[['retweet_count', 'favorite_count', 'compound_score', 'country']]
    df = df.rename(columns={"retweet_count": "retweets", "favorite_count": "likes", "compound_score": "sentiment"})
    df['tweets'] = 1
    df = clean_and_sum(df, 'country')

    df_countries_loc = pd.read_csv('country_loc.csv')

    df_countries = count_countries(data_dict)
    df_countries = df_countries.merge(df_countries_loc, how='left', on='country')
    df_countries = df_countries.dropna()
    df_countries = add_radius(df_countries)
    df_countries = sent_colors(df_countries)
    return df

In [31]:
####general data####


import requests

params=dict(q='energy',limit=100000)
headers={'X-Meili-API-Key':'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}
lexai_url = f"http://35.223.18.2/indexes/twitter_query/search/"
data = requests.get(lexai_url,params=params,headers=headers).json()
data_df = pd.DataFrame(data['hits'])
data_df = data_df.sort_values(by=['timestamp'])
data_dict = data_df.to_dict('records')   #creates dictionary for further use
data_df.shape


(1654, 21)

In [ ]:
#####for mapping by country ####

df_europe = pd.read_csv('list_cities3.csv', delimiter= ';')
list_cities = list(df_europe['city'])
list_countries = list(df_europe['country'])
    
    
df = pd.DataFrame(data_dict)

df = df[['retweet_count', 'favorite_count', 'compound_score', 'user_loc']]
df = df.rename(columns={'user_loc': 'country', "retweet_count": "retweets", "favorite_count": "likes", "compound_score": "sentiment"})
df['tweets'] = 1
def
if ',' in tweet['user_loc']:
            list_loc = tweet['user_loc'].split(',')
            if list_loc[1] in list_countries:
                country_counts['country'].append(list_loc[1])

                
                
df = clean_and_sum(df, 'country')

df_countries_loc = pd.read_csv('country_loc.csv')

    
df= df.merge(df_countries_loc, how='left', on='country')
df = df.dropna()
df = add_radius(df)
df = sent_colors(df)

In [197]:
#####press data#####

params=dict(q='',limit=100000)
headers={'X-Meili-API-Key':'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}
lexai_url = f"http://35.223.18.2/indexes/twitter_press/search/"
data_press = requests.get(lexai_url,params=params,headers=headers).json()
data_press_df=pd.DataFrame(data_press['hits'])
data_press_df = data_press_df.sort_values(by=['timestamp'])
data_press_df = data_press_df[data_press_df["timestamp"] >= 1.622115e+09]
data_press_df.head()
data_press_df.shape

(4590, 22)

In [4]:
###politician data####
import requests
params=dict(q='energy',limit=100000)
headers={'X-Meili-API-Key':'OTkwNzQ0ZGRkZTc0NDcwM2RlMzFlOGIx'}
lexai_url = f"http://35.223.18.2/indexes/twitter_politicians/search/"
data_pol = requests.get(lexai_url,params=params,headers=headers).json()
data_pol_df=pd.DataFrame(data_pol['hits'])
data_pol_df = data_pol_df.sort_values(by=['timestamp'])
data_pol_df = data_pol_df[data_pol_df["timestamp"] >= 1.622115e+09]
data_pol_df.head()
data_dict = data_pol_df.to_dict('records')   #creates dictionary for further use

data_pol_df.shape

(321, 25)

In [9]:
data_pol_df.dtypes

id                   int64
timestamp          float64
user                object
text                object
text_en             object
date                object
lang                object
iso_lang            object
user_verified         bool
followers_count      int64
user_loc            object
user_desc           object
user_desc_en        object
user_image          object
hashtags            object
mentions            object
retweet_count        int64
favorite_count       int64
link                object
compound_score     float64
sentiment           object
name                object
country             object
eu_group            object
national_group      object
dtype: object

In [24]:
df_countries_loc = pd.read_csv('country_loc.csv')

,Unnamed: 0,country,lat,lon
0,0,Germany,51.083420,10.423447
1,1,Spain,39.326068,-4.837979
2,2,Italy,42.638426,12.674297
3,3,Romania,45.985213,24.685923
4,4,France,46.603354,1.888334


In [28]:
df = refine_pol_press(data_dict)
df

<ipython-input-17-e11102f71b4b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'] * (df['retweets']+1) #scores sentiment of a tweet by retweet_count


,country,retweets,likes,sentiment,tweets,Unnamed: 0,lat,lon,radius,r,g,b,s
0,Bulgaria,4,14,2.8,1,13.0,42.607397,25.485662,22000.000000,96,130,253,133.666003
2,Denmark,43,294,2.4,12,21.0,55.670249,10.333328,26557.438524,96,130,253,127.459667
3,Finland,25,296,0.8,5,30.0,63.246778,25.920916,25000.000000,96,130,253,94.721360
4,France,1918,2641,16.2,24,4.0,46.603354,1.888334,63794.976881,96,130,253,251.246118
5,Germany,2589,612,5.6,79,0.0,51.083420,10.423447,70882.216933,96,130,253,168.321596
6,Greece,1355,163,2.6,69,28.0,38.995368,21.987713,56810.324639,96,130,253,130.622577
7,Hungary,0,10,0.1,3,8.0,47.181759,19.506094,20000.000000,96,130,253,65.811388
8,Ireland,8,22,1.1,5,45.0,52.865196,-7.979460,22828.427125,96,130,253,102.440442
9,Latvia,6,7,3.1,2,34.0,56.840649,24.753764,22449.489743,96,130,253,138.034084
10,Netherlands,166,144,2.9,11,18.0,52.500170,5.748082,32884.098727,96,130,253,135.146932


In [13]:
data_slice = data_slice.to_dict('records')

In [30]:
from PIL import Image
image = Image.open('legend.jpg')

image.show()
#st.image(image, caption='Sunrise by the mountains')